# IBM Applied Data Science Capstone Project

Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

To create the dataframe:

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.


2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.


6. Use the .shape method to print the number of rows of your dataframe.

In [1]:
#Import Libraries

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json

#!conda install -c conda-forge geopy --yes
#import geocoder

import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes
#import folium

from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans


### Scrap data from Wikipedia page into a DataFrame

In [16]:
# send the GET request
import random
# Import urllib
import urllib.request

#data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(source,'html.parser')

#
table = soup.find('table')
table_rows = table.find_all('tr')

# create lists to store data
list_data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        list_data.append(row)

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [18]:
df = pd.DataFrame(list_data, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,None
1,M2A,Not assigned,None
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


In [29]:
# Cells that have an assigned borough. 
#And Ignore cells with a borough that is 'Not assigned'.

df_dropna = df[df.Borough != 'Not assigned'].reset_index(drop=True)
df_dropna.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### More than one neighborhood can exist in one postal code area. 
### For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [38]:
# More than one neighborhood can exist in one postal code area.

df_grouped = df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [37]:
#Group
df_grouped.loc[df_grouped['Neighbourhood']=='Not assigned', ['Neighbourhood']]
df_grouped.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Show the data

In [24]:
# Represent the data
df1 = pd.DataFrame(columns=["PostalCode", "Borough", "Neighbourhood"])

test_data_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

for postcode in test_data_list:
    df1 = df1.append(df_grouped[df_grouped["PostalCode"]==postcode], ignore_index=True)
    
df1

,PostalCode,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,Parkview Hill / Woodbine Gardens
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,Wexford / Maryvale
7,M9V,Etobicoke,South Steeles / Silverstone / Humbergate / Jam...
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,CN Tower / King and Spadina / Railway Lands / ...


 
 ### Use the .shape method to print the number of rows of your dataframe.
 

In [26]:
# Print the number of rows of the cleaned dataframe
df_grouped.shape


(103, 3)